# NBA Probability Model

## Setup

In [ ]:
!pip install pyGMs

In [ ]:
import pyGMs as gm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Data


In [ ]:
import sqlite3

# Reading from sqlite file obtained from Kaggle https://www.kaggle.com/wyattowalsh/basketball
db = sqlite3.connect('/content/drive/My Drive/basketball.sqlite') 

# Assigning numbers to teams for referencing
team_abbrevs = db.execute('''SELECT abbreviation FROM Team''')
team_dict = {team[0]:i for i,team in enumerate(team_abbrevs.fetchall())}
team_dict_rev = {v:k for k,v in team_dict.items()}

# Retrieves Game data from sql file, replacing wins and losses with 1,-1 for use with pyGM
# 2019-20 regular season, 2018-19 regular season, 2017-18 regular season, 2019-20 post ASB, 2020-21 regular season
season_start = ['2019-10-22', '2018-10-16', '2017-10-17', '2020-02-16', '2020-12-22']
season_end = ['2020-04-15', '2019-04-10','2018-04-11', '2020-08-14', '2021-05-17']

results = [db.execute('''SELECT GAME_ID, GAME_DATE, TEAM_ABBREVIATION_HOME, WL_HOME, TEAM_ABBREVIATION_AWAY, WL_AWAY FROM Game WHERE GAME_DATE >= '{}' and GAME_DATE <= '{}' '''.format(start_date,end_date)) for start_date,end_date in zip(season_start,season_end)]

previous_seasons = []
for result in results:
  data = np.array(result.fetchall())
  data = np.where(data == 'W', +1,data)
  data = np.where(data == 'L', -1,data)
  previous_seasons.append(data)

## Factors and Modeling

In [ ]:
# Refurbished code from pyGM-skill example
nplayers = 30
nlevels = 10   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability


# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
# This loop will differ based on what data we want to use from previous_seasons
# two_seasons = np.append(previous_seasons[0], previous_seasons[1], 0)
for g in previous_seasons[4]:
    P1,P2,win = team_dict[g[2]], team_dict[g[4]], g[3]
    if win == None:
      continue
    if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [ ]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [ ]:
from pyGMs.factor import *

# Function from pyGM source code, adjusted to fix ufunc 'multiply' error 
def LBP(model, maxIter=100, verbose=False):
    beliefs_F = [ f/f.sum() for f in model.factors ]       # copies & normalizes each f
    beliefs_V = [ Factor([v],1.0/v.states) for v in model.X ] # variable beliefs
    msg = {}
    for f in model.factors:
        for v in f.vars:
            msg[v,f] = Factor([v],1.0)  # init msg[i->alpha]
            msg[f,v] = Factor([v],1.0)  # and  msg[alpha->i]
    
    for t in range(1,maxIter+1):               # for each iteration:
        # Update beliefs and outgoing messages for each factor:
        for a,f in enumerate(model.factors):
            beliefs_F[a] = f.copy()                 # find f * incoming msgs & normalize
            for v in f.vars: 
                beliefs_F[a] = beliefs_F[a]*msg[v,f]
            beliefs_F[a] /= beliefs_F[a].sum()      #   divide by i->f & sum out all but Xi 
            for v in f.vars: msg[f,v] = beliefs_F[a].marginal([v])/msg[v,f]
        # Update beliefs and outgoing messages for each variable:
        for i,v in enumerate(model.X):
            beliefs_V[i] = Factor([v],1.0)       # find product of incoming msgs & normalize
            for f in model.factorsWith(v): 
                beliefs_V[i] = beliefs_V[i] * msg[f,v]
            beliefs_V[i] /= beliefs_V[i].sum()      #   divide by f->i to get msg i->f
            for f in model.factorsWith(v): msg[v,f] = beliefs_V[i]/msg[f,v]
            
        #for f in model.factors:                    # print msgs and beliefs for debugging
        #    for v in f.vars:
        #        print v,"->",f,":",msg[X[v],f].table
        #        print f,"->",v,":",msg[f,X[v]].table
        #for b in beliefs_F: print b, b.table
        #for b in beliefs_V: print b, b.table

        # Compute estimate of the log partition function:
        # E_b [ log f ] + H_Bethe(b) = \sum_f E_bf[log f] + \sum_f H(bf) + \sum (1-di) H(bi)
        lnZ = sum([(1-len(model.factorsWith(v)))*beliefs_V[v].entropy() for v in model.X])
        for a,f in enumerate(model.factors):
            lnZ += (beliefs_F[a] * f.log()).sum()
            lnZ += beliefs_F[a].entropy()
        if verbose: print("Iter "+str(t)+": "+str(lnZ))
    return lnZ,beliefs_V

In [ ]:
#from pyGM.messagepass import LBP, NMF
lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
#lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 1: 20.17953989896577
Iter 2: -573.688572484108
Iter 3: -688.4722142715212
Iter 4: -688.556372583118
Iter 5: -690.1261637371402
Iter 6: -690.4610867092381
Iter 7: -690.4509492994517
Iter 8: -690.4397721745529
Iter 9: -690.4374623009738
Iter 10: -690.4358439580498


## Printing out mean skill estimates

In [ ]:
print("Mean skill estimates: ")
mean_skill = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]
# Teams, sorted based on their skill level 
sort_index = np.argsort(mean_skill)
teams_sorted_skill = [(mean_skill[k],team_dict_rev[k]) for k in sort_index]
teams_sorted_skill = np.flip(np.array(teams_sorted_skill))
print(teams_sorted_skill)

Mean skill estimates: 
[['UTA' '8.026962830506776']
 ['PHX' '7.4805554257096']
 ['PHI' '7.176916710059121']
 ['BKN' '7.007598298635747']
 ['LAC' '6.929677580865192']
 ['DEN' '6.855535995475407']
 ['MIL' '6.625762511706141']
 ['LAL' '6.043512464012362']
 ['DAL' '5.763727096352951']
 ['NYK' '5.723742306270763']
 ['ATL' '5.556716029743207']
 ['POR' '5.526447306832022']
 ['MIA' '5.308048713361209']
 ['MEM' '5.088915027661471']
 ['GSW' '4.733885419536277']
 ['BOS' '4.632573105068779']
 ['WAS' '4.117157362059609']
 ['SAS' '4.059208858139556']
 ['IND' '4.056537866862383']
 ['CHA' '3.9182380142232045']
 ['NOP' '3.6491552192622083']
 ['CHI' '3.5175690198588976']
 ['SAC' '3.249603599472053']
 ['TOR' '2.6258694261424487']
 ['OKC' '1.941814194193427']
 ['MIN' '1.9323897568351986']
 ['CLE' '1.6285238605138836']
 ['ORL' '1.4175607353942432']
 ['DET' '1.2327198479262738']
 ['HOU' '0.9334598586809261']]


## Helper Functions

These functions help us predict games, predict a series between two teams, and predict entire playoff brackets

In [ ]:
def PredictGame(team1, team2, seriesScore, team_dict, Pwin, bel, gm, X):
  i,j = team_dict[team1],team_dict[team2]
  # print("Estimated probability {} beats {} next time:".format(team1,team2))
  home = Pwin if i<j else 1-Pwin
  gameProb = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],home)).table.sum()
  homeWin = np.random.rand() < gameProb

  if homeWin:
    seriesScore[0] += 1
  else:
    seriesScore[1] += 1

In [ ]:
def SimulateSeries(team1, team2, seriesScore, team_dict, Pwin, bel, gm, X):
  seriesScore = [0,0]
  for _ in range (7):
    PredictGame(team1, team2, seriesScore, team_dict, Pwin, bel, gm, X)
    if seriesScore[0] == 4:
      # print(team1)
      return team1
    elif seriesScore[1] == 4: 
      # print(team2)
      return team2

In [ ]:
# teams, round1/2/3_winners is used for prediction of the 2020 playoffs (which we know results to)
# teams2021 is used for prediction of 2021 playoffs and predicting a winner
teams = ['LAL', 'POR', 'HOU', 'OKC', 'LAC', 'DAL', 'DEN', 'UTA', 'MIL', 'ORL', 'IND', 'MIA', 'TOR', 'BKN', 'BOS', 'PHI']
teams2021 = ['UTA', 'MEM', 'LAC', 'DAL', 'DEN', 'POR', 'PHX', 'LAL', 'PHI', 'WAS', 'NYK', 'ATL', 'MIL', 'MIA', 'BKN', 'BOS']
round1_winners = {'LAL','HOU','LAC','DEN','MIL','MIA','TOR','BOS'}
round2_winners = {'LAL','DEN','MIA','BOS'}
round3_winners = {'LAL','MIA'}
champion = 'LAL'
def SimulateBracket(teams, seriesScore, team_dict, Pwin, bel, gm, X, round1_winners, round2_winners, round3_winners, champion, counts, count):
  if len(teams) == 1:
    count += teams[0] == champion
    counts.append(count)
    
    return teams[0]
  winners = []
  for i in range(0, len(teams), 2):
    winner = SimulateSeries(teams[i], teams[i+1], seriesScore, team_dict, Pwin, bel, gm, X)
    winners.append(winner)

    winners_set = set(winners)

  if len(winners) == 8:
    count += len(winners_set.intersection(round1_winners))
  elif len(winners) == 4:
    count += len(winners_set.intersection(round2_winners))
  elif len(winners) == 2:
    count += len(winners_set.intersection(round3_winners))
    
  # print(winners)
  return SimulateBracket(winners, seriesScore, team_dict, Pwin, bel, gm, X, round1_winners, round2_winners, round3_winners, champion, counts, count)


## Simulating a bracket 1000 times

We simulate a playoff bracket 100 times see the average amount of series predicted by the model (max 15 series).

In [ ]:
total = 0
counts = []
for _ in range(1000):
  count = 0
  SimulateBracket(teams, seriesScore, team_dict, Pwin, bel, gm, X, round1_winners, round2_winners, round3_winners, champion,counts, count)

print(np.mean(counts))

5.449


# 2021 Playoff Predictions
The next section predicts the champions for this year using the data from regular season, which was our most accurate model for predicting correct series for the 2020 season.

In [ ]:
predicted_champions = dict()
for _ in range(1000):
  champion = SimulateBracket(teams2021, seriesScore, team_dict, Pwin, bel, gm, X, round1_winners, round2_winners, round3_winners, champion,counts, count)
  if champion not in predicted_champions.keys():
    predicted_champions[champion] = 1
  else:
    predicted_champions[champion] += 1

In [ ]:
print(predicted_champions)

{'UTA': 274, 'PHI': 192, 'PHX': 159, 'NYK': 22, 'MIL': 52, 'BKN': 134, 'DEN': 55, 'DAL': 7, 'LAC': 71, 'BOS': 3, 'LAL': 16, 'ATL': 9, 'MEM': 2, 'MIA': 2, 'POR': 2}
